<h1 style=color:#3498DB>ImageDataGenerator with Horses and Humans dataset</h1>

In [20]:
import os
import wget
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop

<h3 style=color:#3498DB>Download dataset</h3>

In [2]:
wget.download('https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip')

wget.download('https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip')

100% [........................................................................] 11480187 / 11480187

'validation-horse-or-human.zip'

<h3 style=color:#3498DB>Unzip dataset</h3>

In [7]:
# Unzip training set
local_zip = 'horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp\\horse-or-human')

# Unzip validation set
local_zip = 'validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp\\validation-horse-or-human')

zip_ref.close()

<h3 style=color:#3498DB>Directories</h3>

In [12]:
# Directory with training horse pictures
train_horse_dir = os.path.join('tmp\\horse-or-human\\horses')

# Directory with training human pictures
train_human_dir = os.path.join('tmp\\horse-or-human\\humans')

# Directory with validation horse pictures
validation_horse_dir = os.path.join('tmp\\validation-horse-or-human\\horses')

# Directory with validation human pictures
validation_human_dir = os.path.join('tmp\\validation-horse-or-human\\humans')

In [13]:
print(f'total training horse images: {len(os.listdir(train_horse_dir))}')
print(f'total training human images: {len(os.listdir(train_human_dir))}')
print(f'total validation horse images: {len(os.listdir(validation_horse_dir))}')
print(f'total validation human images: {len(os.listdir(validation_human_dir))}')

total training horse images: 500
total training human images: 527
total validation horse images: 128
total validation human images: 128


<h1 style=color:#3498DB>ImageDataGenerator</h1>

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)


# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                                                    'tmp\\horse-or-human',
                                                    target_size=(300, 300),
                                                    batch_size=128,
                                                    class_mode='binary')


# Flow validation images in batches of 128 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                                                              'tmp\\validation-horse-or-human',
                                                              target_size=(300, 300),
                                                              batch_size=32,
                                                              class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


<h3 style=color:#3498DB>Create Model</h3>

In [21]:
def createmodel():
    model = keras.models.Sequential([
                                     # Note the input shape is the desired size of the image 300x300 with 3 bytes color
                                     keras.Input(shape=(300, 300, 3)),

                                     # This is the first convolution
                                     keras.layers.Conv2D(16, (3,3), activation='relu'),
                                     keras.layers.MaxPooling2D(2, 2),

                                     # The second convolution
                                     keras.layers.Conv2D(32, (3,3), activation='relu'),
                                     keras.layers.MaxPooling2D(2,2),

                                     # The third convolution
                                     keras.layers.Conv2D(64, (3,3), activation='relu'),
                                     keras.layers.MaxPooling2D(2,2),

                                     # The fourth convolution
                                     keras.layers.Conv2D(64, (3,3), activation='relu'),
                                     keras.layers.MaxPooling2D(2,2),

                                     # The fifth convolution
                                     keras.layers.Conv2D(64, (3,3), activation='relu'),
                                     keras.layers.MaxPooling2D(2,2),


                                     # Flatten the results to feed into a DNN
                                     keras.layers.Flatten(),

                                     # 512 neuron hidden layer
                                     keras.layers.Dense(512, activation='relu'),

                                     # Only 1 output neuron. It will contain a value from 0-1 
                                     # where 0 for 1 class ('horses') and 1 for the other ('humans')
                                     keras.layers.Dense(1, activation='sigmoid')
                                     ])
    
    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])
    
    return model

In [22]:
model = createmodel()
history = model.fit(
                    train_generator,
                    steps_per_epoch=8,  
                    epochs=15,
                    verbose=1,
                    validation_data = validation_generator,
                    validation_steps=8)

Epoch 1/15
8/8 [==============================] - 95s 12s/step - loss: 0.7914 - accuracy: 0.5566 - val_loss: 0.6442 - val_accuracy: 0.5156
Epoch 2/15
8/8 [==============================] - 87s 11s/step - loss: 1.1913 - accuracy: 0.6452 - val_loss: 0.8094 - val_accuracy: 0.5000
Epoch 3/15
8/8 [==============================] - 87s 11s/step - loss: 0.6454 - accuracy: 0.7297 - val_loss: 0.5035 - val_accuracy: 0.7461
Epoch 4/15
8/8 [==============================] - 79s 10s/step - loss: 0.4022 - accuracy: 0.8687 - val_loss: 0.5120 - val_accuracy: 0.8789
Epoch 5/15
8/8 [==============================] - 85s 10s/step - loss: 0.3037 - accuracy: 0.8906 - val_loss: 0.7522 - val_accuracy: 0.8594
Epoch 6/15
8/8 [==============================] - 90s 13s/step - loss: 0.2299 - accuracy: 0.9132 - val_loss: 0.9351 - val_accuracy: 0.8320
Epoch 7/15
8/8 [==============================] - 91s 11s/step - loss: 0.2106 - accuracy: 0.9277 - val_loss: 1.0675 - val_accuracy: 0.8398
Epoch 8/15
8/8 [===========

<h3 style=color:#3498DB>Model Summary</h3>

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

<h3 style=color:#3498DB>Save Model</h3>

In [24]:
model.save('tmp\\HorsesHumansmodel.h5')